## EECS 442/504 PS7: Object Detetction

__Please provide the following information__
(e.g. Andrew Owens, ahowens):

Bernardo Bianco Prado, beprado

__Important__: after you download the .ipynb file, please name it as __\<your_uniquename\>_\<your_umid\>.ipynb__ before you submit it to canvas. Example: adam_01101100.ipynb.

**Acknowledgement**: A part of this assignment is adapted from EECS 598/498 Deep Learning for Computer Vision taught by Prof. Justin Johnson.

**Inference Components and Evaluation Metrics for Object Detection**

In this part of the assignment, you will implement the inference and evaluation metrics required for Object Detection. 

You will be implementing **IoU**, **NMS** and **mAP**

In [1]:
# We will import some libraries we need
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data._utils.collate import default_collate
from torch import optim
from torchvision import transforms
import torchvision
from torchvision import models
from torchvision.models import feature_extraction
from torchvision.ops import nms

import matplotlib.pyplot as plt
import numpy as np
import os
import time
import random
from collections import Counter
!pip -q install torchmetrics

     |████████████████████████████████| 529 kB 5.0 MB/s 


We will use GPUs to accelerate our computation in this notebook. Run the following to make sure GPUs are enabled:

In [2]:
if torch.cuda.is_available():
    print('Good to go!')
    DEVICE = torch.device("cuda")
else:
    print('Please set GPU via Edit -> Notebook Settings.')
    DEVICE = torch.device("cpu")

Good to go!


In [3]:
def reset_seed(number):
    """
    Reset random seed to the specific number

    Inputs:
    - number: A seed number to use
    """
    random.seed(number)
    torch.manual_seed(number)
    return

In [4]:
def rel_error(x, y, eps=1e-10):
    """
    Compute the relative error between a pair of tensors x and y,
    which is defined as:

                            max_i |x_i - y_i]|
    rel_error(x, y) = -------------------------------
                      max_i |x_i| + max_i |y_i| + eps

    Inputs:
    - x, y: Tensors of the same shape
    - eps: Small positive constant for numeric stability

    Returns:
    - rel_error: Scalar giving the relative error between x and y
    """
    """ returns relative error between x and y """
    top = (x - y).abs().max().item()
    bot = (x.abs() + y.abs()).clamp(min=eps).max().item()
    return top / bot

## Intersection Over Union (IoU)

The definition of IoU and instructions on how to compute IoU can be found in the lecture 11 slides (47-48) and discussion 09 slides:
https://www.eecs.umich.edu/courses/eecs442-ahowens/fa22/slides/lec11-object.pdf

Implement the `intersection_over_union` function given below. We will then compare your implementation of IoU with a toy example. We will later reuse your IoU implementation for computing mean average precision (mAP).

In [5]:
def intersection_over_union(boxes1: torch.Tensor, boxes2: torch.Tensor) -> torch.Tensor:
    """
    Compute intersection-over-union (IoU) between pairs of box tensors. Input
    box tensors must in XYXY format.

    Args:
        boxes1: Tensor of shape `(M, 4)` giving a set of box co-ordinates.
        boxes2: Tensor of shape `(N, 4)` giving another set of box co-ordinates.

    Returns:
        torch.Tensor
            Tensor of shape (M, N) with `iou[i, j]` giving IoU between i-th box
            in `boxes1` and j-th box in `boxes2`.
    """

    ##########################################################################
    # TODO: Implement the IoU function here.   
    # Hint: You can  use torch.max and torch.min here  
    # Remember to account for the case that the boxes do not intersect
    # You may use a for loop if necessary, but vectorized code is always better
    ##########################################################################
    
    # reshape tensors for vectorized code
    M = boxes1.shape[0]
    N = boxes2.shape[0]
    b1 = boxes1.view(-1,1,4).repeat((1,N,1))
    b2 = boxes2.view(1,-1,4).repeat((M,1,1))
    # compute the box coordinates of the intersection of boxes
    top_left = torch.max(b1[:,:,0:2], b2[:,:,0:2])
    bottom_right = torch.min(b1[:,:,2:], b2[:,:,2:])
    width = bottom_right[:,:,0] - top_left[:,:,0]
    height = bottom_right[:,:,1] - top_left[:,:,1]
    # set negative lengths to zero (non-intersecting boxes)
    width[width < 0] = 0
    height[height < 0] = 0
    # area of intersection
    intersection = width * height
    # area of union
    area_box1 = (b1[:,:,2] - b1[:,:,0]) * (b1[:,:,3] - b1[:,:,1])
    area_box2 = (b2[:,:,2] - b2[:,:,0]) * (b2[:,:,3] - b2[:,:,1])
    union = area_box1 + area_box2 - intersection

    iou = intersection / union

    ##########################################################################
    #                             END OF YOUR CODE                           #
    ##########################################################################
    return iou

In [6]:
boxes1 = torch.Tensor([[10, 10, 90, 90], [20, 20, 40, 40], [60, 60, 80, 80]])
boxes2 = torch.Tensor([[10, 10, 90, 90], [60, 60, 80, 80], [30, 30, 70, 70]])

expected_iou = torch.Tensor(
    [[1.0, 0.0625, 0.25], [0.0625, 0.0, 0.052631579], [0.0625, 1.0, 0.052631579]]
)
result_iou = intersection_over_union(boxes1, boxes2)

print("Relative error:", rel_error(expected_iou, result_iou))

Relative error: 0.0


## Non-Maximum Suppression (NMS)

The definition of NMS and instructions on how to compute NMS can be found in the lecture 11 slides (47-48) and discussion 09 slides:
https://www.eecs.umich.edu/courses/eecs442-ahowens/fa22/slides/lec11-object.pdf

Implement the `nms` function given below. We will then compare your implementation of NMS with the implementation in torchvision. Most likely, your implementation will be faster on CPU than on CUDA, and the torchvision implementation will likely be much faster than yours.
This is expected, but your implementation should produce the same outputs as the torchvision version.

In [7]:
def nms(boxes: torch.Tensor, scores: torch.Tensor, iou_threshold: float = 0.5):
    """
    Non-maximum suppression removes overlapping bounding boxes.

    Args:
        boxes: Tensor of shape (N, 4) giving top-left and bottom-right coordinates
            of the bounding boxes to perform NMS on.
        scores: Tensor of shpe (N, ) giving scores for each of the boxes.
        iou_threshold: Discard all overlapping boxes with IoU > iou_threshold

    Returns:
        keep: torch.long tensor with the indices of the elements that have been
            kept by NMS, sorted in decreasing order of scores;
            of shape [num_kept_boxes]
    """

    if (not boxes.numel()) or (not scores.numel()):
        return torch.zeros(0, dtype=torch.long)

    keep = None
    #############################################################################
    # TODO: Implement non-maximum suppression which iterates the following:     #
    #       1. Select the highest-scoring box among the remaining ones,         #
    #          which has not been chosen in this step before                    #
    #       2. Eliminate boxes with IoU > threshold                             #
    #       3. If any boxes remain, GOTO 1                                      #
    #       Your implementation should not depend on a specific device type;    #
    #       you can use the device of the input if necessary.                   #
    # HINT: You can refer to the torchvision library code:                      #
    # github.com/pytorch/vision/blob/main/torchvision/csrc/ops/cpu/nms_kernel.cpp
    #############################################################################
    
    sorted, indices = torch.sort(scores, descending=True)
    sorted_boxes = boxes[indices]
    keep_indices = torch.ones(sorted.shape, dtype=torch.bool)
    for i in range(sorted.shape[0]):
      if keep_indices[i] == False:
        continue
      iou = intersection_over_union(sorted_boxes[i], sorted_boxes[i+1:])[0]
      thresh = iou <= iou_threshold
      keep_indices = torch.cat((keep_indices[:i+1], thresh))
    keep = indices[keep_indices]
    
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    return keep

The difference in your implementation of nms and the nms from the torchvision package should be in the order of `1e-3` or lesser

In [8]:
#Sanity check

reset_seed(0)


boxes = (100.0 * torch.rand(5000, 4)).round()
boxes[:, 2] = boxes[:, 2] + boxes[:, 0] + 1.0
boxes[:, 3] = boxes[:, 3] + boxes[:, 1] + 1.0
scores = torch.randn(5000)

names = ["your_cpu", "torchvision_cpu", "torchvision_cuda"]
iou_thresholds = [0.3, 0.5, 0.7]
elapsed = dict(zip(names, [0.0] * len(names)))
intersects = dict(zip(names[1:], [0.0] * (len(names) - 1)))

for iou_threshold in iou_thresholds:
    tic = time.time()
    my_keep = nms(boxes, scores, iou_threshold)
    elapsed["your_cpu"] += time.time() - tic

    tic = time.time()
    tv_keep = torchvision.ops.nms(boxes, scores, iou_threshold)
    elapsed["torchvision_cpu"] += time.time() - tic
    intersect = len(set(tv_keep.tolist()).intersection(my_keep.tolist())) / len(tv_keep)
    intersects["torchvision_cpu"] += intersect

    tic = time.time()
    tv_cuda_keep = torchvision.ops.nms(boxes.to(device=DEVICE), scores.to(device=DEVICE), iou_threshold).to(
        my_keep.device
    )
    torch.cuda.synchronize()
    elapsed["torchvision_cuda"] += time.time() - tic
    intersect = len(set(tv_cuda_keep.tolist()).intersection(my_keep.tolist())) / len(
        tv_cuda_keep
    )
    intersects["torchvision_cuda"] += intersect

for key in intersects:
    intersects[key] /= len(iou_thresholds)

# You should see < 1% difference
print("Testing NMS:")
print("Your        CPU  implementation: %fs" % elapsed["your_cpu"])
print("torchvision CPU  implementation: %fs" % elapsed["torchvision_cpu"])
print("torchvision CUDA implementation: %fs" % elapsed["torchvision_cuda"])
print("Speedup CPU : %fx" % (elapsed["your_cpu"] / elapsed["torchvision_cpu"]))
print("Speedup CUDA: %fx" % (elapsed["your_cpu"] / elapsed["torchvision_cuda"]))
print(
    "Difference CPU : ", 1.0 - intersects["torchvision_cpu"]
)  # in the order of 1e-3 or less
print(
    "Difference CUDA: ", 1.0 - intersects["torchvision_cuda"]
)  # in the order of 1e-3 or less

Testing NMS:
Your        CPU  implementation: 11.580528s
torchvision CPU  implementation: 0.090222s
torchvision CUDA implementation: 5.444486s
Speedup CPU : 128.355412x
Speedup CUDA: 2.127019x
Difference CPU :  0.0019011406844106071
Difference CUDA:  0.00189753320683117


## Mean Average Precision (mAP)

The definition of mAP and instructions on how to compute mAP can be found in the lecture 11 slides (47-48) and discussion 09 slides:
https://www.eecs.umich.edu/courses/eecs442-ahowens/fa22/slides/lec11-object.pdf

Implement the `mean_average_precision` function given below. We will then compare your implementation of mAP with a toy example. Make sure the relative error you get is below 1e-5

In [82]:
from sys import breakpointhook
def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=20
):
    """
    Calculates mean average precision 

    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct bounding boxes  
        specified as [train_idx, class_prediction, x1, y1, x2, y2]
        iou_threshold (float): threshold where predicted bboxes is correct
        num_classes (int): number of classes

    Returns:
        float: mAP value across all classes given a specific IoU threshold 
    """

    # list storing all AP for respective classes
    average_precisions = []

    for c in range(num_classes):
      detections = []
      ground_truths = []
      ##########################################################################
      # TODO: Implement the mAP function here.  
      # 1. For each class, find the ground truths and predicted boxes for that class and fill the list detections, ground_truths
      # 2. Sort the predicted boxes by class scores
      # 3. Find the number of ground truth bounding boxes for each image
      # 4. Keep track of all the GT boxes covered. If a predicted box is counted as True Positive based off a particular GT box, 
      #    this GT box should not be matched to any other predicted box thereafter
      # 5. For each detection, calculate IoU with gt boxes of the same image and remember to account for the gt boxes that have already been covered                           
      ##########################################################################

      # select classes and sort by score
      class_true_boxes, class_pred_boxes = true_boxes[true_boxes[:,1] == c], pred_boxes[pred_boxes[:,1] == c]
      sorted_scores, sorted_indices = torch.sort(class_pred_boxes[:,2], descending=True)
      class_pred_boxes = class_pred_boxes[sorted_indices]
      # useful variables for precision and recall
      precisions, recalls = [], []
      total_detections, total_test_instances = sorted_indices.shape[0], class_true_boxes.shape[0]
      gt_box_used = torch.zeros(total_test_instances, dtype=torch.bool)
      # compute the intersection over union of the classes
      iou = intersection_over_union(class_pred_boxes[:,-4:], class_true_boxes[:,-4:])
      # edge cases for when the denominator is 0
      if total_test_instances == 0:
        precisions.append(1 if total_detections == 0 else 0)
        recalls.append(1)
        precisions = torch.tensor(precisions)
        recalls = torch.tensor(recalls)
        continue
      # compute precision and recall
      true_positive = 0
      for i in range(total_detections):
        for j in range(total_test_instances):
          if class_true_boxes[j,0] == class_pred_boxes[i,0] and iou[i,j] > iou_threshold and not gt_box_used[j]:
            true_positive += 1
            gt_box_used[j] = True
            break
        precisions.append(true_positive / (i+1))
        recalls.append(true_positive / total_test_instances)

      precisions = torch.tensor(precisions)
      recalls = torch.tensor(recalls)
   
      ##########################################################################
      #                             END OF YOUR CODE                           #
      ##########################################################################
      # precisions - y values, recalls - x values - we append 1 to the precisions since we want to start at 1
      # for numerical intergation and similarly we append 0 to recalls since we want to start at 0 for numerical integration
      precisions = torch.cat((torch.tensor([1]), precisions))
      recalls = torch.cat((torch.tensor([0]), recalls))  
      # torch.trapz for numerical integration
      #We do this to calculate the area under the precision - recall curve
      average_precisions.append(torch.trapz(precisions, recalls))
    map=sum(average_precisions) / len(average_precisions)

    return map

In [83]:
pred_boxes = torch.Tensor([[0,0,0.536,258.0, 41.0, 606.0, 285.0],[1,3,0.318,61.0, 22.75, 565.0, 632.42],[1,2,0.725,12.66,3.32,281.26,275.23]])
gt_boxes = torch.Tensor([[0,0,214.0, 41.0, 562.0, 285.0],[1,2,13.00, 22.75, 548.98, 632.42],[1,2,1.66,3.32,270.26,275.23]])
num_classes=20
iou_thresholds=torch.arange(0.5,0.96,0.05)

from torchmetrics.detection.mean_ap import MeanAveragePrecision
#MeanAveragePrecision has the foll structure preds=[dict for img1, dict for img2 ....], target=[dict for img1, dict for img2 ....]
preds = [
    dict(
        boxes=torch.tensor([[258.15, 41.29, 606.41, 285.07]]),
         scores=torch.tensor([0.236]),
         labels=torch.tensor([4]),
         ),
    dict(
         boxes=torch.tensor([[61.0, 22.75, 565.0, 632.42],[12.66,3.32,281.26,275.23]]),
         scores=torch.tensor([0.318,0.725]),
         labels=torch.tensor([3,2]),
        )
    ]
target = [
    dict(
        boxes=torch.tensor([[214.15, 41.29, 562.41, 285.07]]),
         labels=torch.tensor([4]),
         ),
    dict(
        boxes=torch.tensor([[13.00, 22.75, 548.98, 632.42],[1.66,3.32,270.26,275.23]]),
         labels=torch.tensor([2,2]),
         )
    ]

metric = MeanAveragePrecision()
metric.update(preds, target)
from pprint import pprint
expected_map=metric.compute()["map"]

result_map=0
for iou_thresh in iou_thresholds:
  iou_thresh=round(iou_thresh.item(),2)
  result_map+=mean_average_precision(pred_boxes=pred_boxes,true_boxes=gt_boxes,num_classes=num_classes,iou_threshold=iou_thresh)
result_map=result_map/iou_thresholds.shape[0]

print("resulting mAP", result_map.item())

print("Relative error:", rel_error(expected_map, result_map))

resulting mAP 0.5249999761581421
Relative error: 0.002117149665685782


# Convert Notebook to PDF

[Alternative if the cell below doesn't work.](https://docs.google.com/document/d/1QTutnoApRow8cOxNrKK6ISEkA72QGfwLFXbIcpvarAI/edit?usp=sharing)

In [84]:
import os
from google.colab import drive
from google.colab import files

drive_mount_point = '/content/drive/'
drive.mount(drive_mount_point)

Mounted at /content/drive/


In [85]:
# generate pdf
# Please provide the full path of the notebook file below
# Important: make sure that your file name does not contain spaces!

# Ex: notebookpath = '/content/drive/My Drive/Colab Notebooks/EECS_442_PS4_FA_2022_Starter_Code.ipynb'
#notebookpath = '/content/drive/MyDrive/PS7_FCOS/PS7_EECS_504_part2.ipynb' 
notebookpath = '/content/drive/My Drive/Colab Notebooks/PS7_504_beprado.ipynb' 

file_name = notebookpath.split('/')[-1]
get_ipython().system("apt update && apt install texlive-xetex texlive-fonts-recommended texlive-generic-recommended")
get_ipython().system("jupyter nbconvert --to PDF {}".format(notebookpath.replace(' ', '\\ ')))
files.download(notebookpath.split('.')[0]+'.pdf')

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,332 kB]
Hit:14 http://ppa.launchpad.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>